In [ ]:
## OpenSoundscape RIBBIT tutorial
# http://opensoundscape.org/en/latest/tutorials/RIBBIT_pulse_rate_demo.html

## Setup 

In [13]:
# suppress warnings
import warnings
warnings.simplefilter('ignore')
warnings.simplefilter('ignore', UserWarning)

#import packages
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import subprocess
from datetime import datetime


#local imports from opensoundscape
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from opensoundscape.ribbit import ribbit

# create big visuals
plt.rcParams['figure.figsize']=[15,8]
pd.set_option('display.precision', 2)

## Set parameters 

In [ ]:
# These parameters can be adjusted to try to improve the model or to change the model for a different herp species 

In [14]:
# minimum and maximum rate of pulsing (pulses per second) to search for
pulse_rate_range = [14,28] 

# look for a vocalization in the range of 600-800 Hz
signal_band = [300,1000]

# subtract the amplitude signal from these frequency ranges
noise_bands = [[150,200], [1300,1600]]

#divides the signal into segments this many seconds long, analyzes each independently
clip_duration = 1.5 #seconds
clip_overlap = 0.5 #seconds

#if True, it will show the power spectrum plot for each audio segment
show_plots = False 


## Analyzing verified files

In [15]:
# run RIBBIT for a specific every index in a df where the index is a file path; save the score in that df 
def run_ribbit_for_indices(df):
    start_time = datetime.now() # track how long the model takes
    checkpoint = 0; # create checkpoint for saving RIBBIT scores periodically 

    # for every data file for year i, pond j... calculate RIBBIT score 
    for path in df.index:

        try: # make sure RIBBIT is working for this audio file, if it doesn't work for some reason, skip this so the model will continue running 
            #make the spectrogram
            spec = Spectrogram.from_audio(Audio.from_file(path))

            #run RIBBIT on the spectrogram 
            score_df =  ribbit(
                                spec,
                                pulse_rate_range=pulse_rate_range,
                                signal_band=signal_band,
                                clip_duration=clip_duration,
                                noise_bands=noise_bands,
                                plot=False)


            maxIndex= score_df.score.idxmax() # find the index for where the max RIBBIT score occurs for the audio file 

            # record the max RIBBIT score for the auio file (multiplied by 10,000 to make it easier to read)
            df.at[path,'score'] = score_df.at[maxIndex, 'score'] * 10000
            # record the time in the audio file that the max RIBBIT score occurs (so can be checked manually)
            df.at[path,'time_stamp'] = score_df.at[maxIndex, 'start_time']

            checkpoint = checkpoint + 1 

        except (RuntimeError, EOFError) as runtime_error: # if the file does not work with RIBBIT, skip it and output a failure message; use -1 for RIBBIT score and time stamp
            print("File failed: ", path)
            print(runtime_error)


            df.at[path,'score'] = -1 
            df.at[path,'time_stamp'] = -1
            checkpoint = checkpoint + 1
            pass 

        # every 100 files, print checkpoint message and save RIBBIT scores to csv file to back them up 
        if checkpoint % 100 == 0:
            print("Finished file number ", checkpoint, "/nFile name: ", path)
            print(df.sort_values(by='score',ascending=False).head(1))
            df.to_csv(csv_file_name) 


    df.sort_values(by='score',ascending=False).to_csv(csv_file_name) # sort by highest RIBBIT scores and save to csv
    print('Time elapsed ',datetime.now() - start_time) # print time taken for this pond-year

    #print("Files sorted by score, from highest to lowest:")
    df.sort_values(by='score',ascending=False)   

        

In [12]:
warnings.filterwarnings("ignore")

ich_verified_df = pd.read_csv("./ich_verified_files.csv", index_col = 1)

all_data = list() # setup variable to save RIBBIT scores 

start_time = datetime.now() # track how long the model takes

# for every data file that has been verified... calculate RIBBIT score 
run_ribbit_for_indices(ich_verified_df)

ich_verified_df.sort_values(by='score',ascending=False).to_csv('./ribbit_scores/ichaway_verified_files_ribbit_scores.csv') # sort by highest RIBBIT scores and save to csv
print('Time elapsed ',datetime.now() - start_time) # print time taken for this pond-year

#print("Files sorted by score, from highest to lowest:")
ich_verified_df.sort_values(by='score',ascending=False)

    


/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

...

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 93, using nperseg = 93
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 94, using nperseg = 94
  .format(nperseg, input_length))
/Users/mayalapp/opt/anaconda3/envs/opensoundscape060/lib/python3.7/site-packages/scipy/s

Time elapsed  0:29:46.881690


,Unnamed: 0,year,pond,date,Lcapito,score,time_stamp
file_path,,,,,,,
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/4a/20150224_000100.wav,1059,2015,4a,2015-02-24 00:01:00,1.0,5.06e+02,49.5
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/5a/20150203_194500.wav,4602,2015,5a,2015-02-03 19:45:00,1.0,4.13e+02,175.5
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/4a/20150223_230100.wav,1061,2015,4a,2015-02-23 23:01:00,1.0,3.26e+02,109.5
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/5a/20150306_191000.wav,4605,2015,5a,2015-03-06 19:10:00,1.0,2.95e+02,196.5
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/5a/20150205_194700.wav,4603,2015,5a,2015-02-05 19:47:00,0.0,2.60e+02,157.5
...,...,...,...,...,...,...,...
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/7a/20150308_201100.wav,694,2015,7a,2015-03-08 20:11:00,0.0,1.33e-02,166.5
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/9a/20150308_191100.wav,2476,2015,9a,2015-03-08 19:11:00,0.0,9.04e-03,91.5
/Volumes/Expansion/Frog Call Project/Calling Data/Frog Calls/ichaway_2015/7a/20150308_191100.wav,702,2015,7a,2015-03-08 19:11:00,0.0,3.67e-04,133.5
